# Practical part for the PDC project

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

## Functions handling bit vectors

In [2]:
# Generates a 78-character ASCII string at random.
def generate_ascii_strings():
    return "".join([chr(int(np.random.rand()*127)) for i in range(0, 78)])

# Performs a conversion from ASCII string to binary symbols
def ascii_str_to_binary(ascii_str):
    return str(''.join(format(ord(i), '08b')[1:] for i in ascii_str))

# Performs the inverse conversion, from a binary stream to ASCII characters
def binary_to_ascii_str(binstr):
    byte_binstr = "".join([f"0{byte}" for byte in split_bit_str(binstr, 7)])
    binary_int = int(byte_binstr, 2)
    byte_number = binary_int.bit_length() + 7 // 8
    binary_array = binary_int.to_bytes(byte_number, "big")
    return binary_array.decode()

# Splits a bit string composed of 7-bit codewords into chunks of size 'chunk_size'.
def split_bit_str(bit_str, chunk_size):
    return [bit_str[i:i+chunk_size] for i in range(0, len(bit_str), chunk_size)]

# Gets the whole alphabet from a codeword length
def get_alphabet_from_codeword_length(length):
    return [f'{i:0{length}b}' for i in range(0, 2**length)]

## Decoding criterion functions

In [3]:
# Computes the distance between a symbol and every point of a given constellation and returns the index of the nearest point in the constellation
def minimum_distance_criterion(constellation, symbol):
    return np.argmin([((symbol.real - point.real)**2 + (symbol.imag - point.imag)**2) for point in constellation])

## Debugging functions to check the encoding/decoding states

In [4]:
# Gets every possible codeword lengths for which the length divides the total length of the original bitstring
def compute_codeword_lengths(bit_str):
    n = len(bit_str)
    return [i for i in range(1, n+1) if n % i == 0]

# Gets all the positions where bit_str1 is different from bit_str2 (bit_str1 and bit_str2 MUST HAVE THE SAME SIZE !)
def get_diff_positions(bit_str1, bit_str2):
    return [i for i in range(0, len(bit_str1)) if bit_str1[i] != bit_str2[i]]

## Constellation-related functions

### m-QAM

In [5]:
# Computes the PAM constellation for a given number of points and a distance for each of these points.
def pam(n, d):
    right_side = np.array([d*i + (d/2) for i in range(0, int(n/2))])
    left_side = -right_side
    return np.append(np.sort(left_side), right_side)

# Computes the whole m-QAM constellation, to be used to map each codeword to a point generated here
# WARNING: m MUST BE A POWER OF 2 HERE !!!
def m_qam(m, d, max_energy = (3/2)):
    if d > max_energy:
        raise OverflowError("ERROR: the distance you have set is too big for the project.")

    # First we handle the weird configurations
    if m == 2:
        return pam(m, d)

    elif m == 8:
        real_axis_pam = pam(4, d)
        imaginary_axis_pam = pam(2, d)
        return np.array([[complex(re, im) for re in real_axis_pam] for im in imaginary_axis_pam]).flatten()

    else:    
        # First we take the square root of the QAM size, if it is an integer then we will generate a perfect square
        # Otherwise, we try to find the nearest perfect square with sides of even lengths such that it contains more points than our desired constellation
        axis_len = int(np.sqrt(m))

        if axis_len**2 != m:
             # We want axes containing an even number of points, and a bigger constellation than the one we requested in this case
            if axis_len % 2 != 0:
                axis_len += 1
            else:
                axis_len += 2

        # We compute the difference between the number of points in the nearest perfect square and the number of points in our requested constellation
        nb_points_to_delete = abs(m - axis_len**2)

        # if the difference is positive => we need to remove 'diff_with_nearest_square' points from the nearest perfect square
        # => we generate the perfect square before removing points inside it
        real_axis_pam = pam(axis_len, d)
        if real_axis_pam.max() > max_energy:
            raise OverflowError("ERROR: You have some points that are outside the constrained square. Try to lower the distance or reduce the size of the constellation.")
        
        imaginary_axis_pam = pam(axis_len, d)[::-1]
        points_pairs = np.array([[complex(re, im) for re in real_axis_pam] for im in imaginary_axis_pam])

        # Now that the constellation has been generated, if diff_with_nearest_square is not 0 we remove the additional points
        if nb_points_to_delete > 0:
            nb_points_to_remove_per_quarter = nb_points_to_delete / 4
            nb_rows_to_affect_per_quarter = int(np.sqrt(nb_points_to_remove_per_quarter))

            # If the square root isn't whole => we must re-organize the zones where to delete the points (they are not squares anymore)
            difference_from_whole_square = nb_points_to_remove_per_quarter - nb_rows_to_affect_per_quarter**2

            if difference_from_whole_square > 0:
                nb_rows_to_affect_per_quarter = difference_from_whole_square
                nb_points_to_remove_per_quarter_row = nb_points_to_remove_per_quarter / difference_from_whole_square
            else:
                nb_points_to_remove_per_quarter_row = nb_rows_to_affect_per_quarter

            # First we set the points to delete to 0
            for j in range(0, int(nb_rows_to_affect_per_quarter)):
                for i in range(0, int(nb_points_to_remove_per_quarter_row)):
                    points_pairs[j, i] = 0
                    points_pairs[j, len(points_pairs[0])-1-i] = 0
                    points_pairs[len(points_pairs[0])-1-j, i] = 0
                    points_pairs[len(points_pairs[0])-1-j, len(points_pairs[0])-1-i] = 0

        # We flatten the array to make it easier to map the codewords
        points_pairs = points_pairs.flatten()

        # Then, we remove all the points set to 0 (impossible to have if the dimension is a square with sides of even length, so no worries)
        points_pairs = np.delete(points_pairs, np.argwhere(points_pairs == 0.0+0.0j))

        return points_pairs

# Gets the maximum distance for a given QAM configuration
def get_max_qam_distance(m, tolerance, max_energy):
    if m == 2:
        pam_size = 2
    elif m == 8:
        pam_size = 4
    else:
        pam_size = int(np.sqrt(m))
        if pam_size**2 != m:
            # We want axes containing an even number of points, and a bigger constellation than the one we requested in this case
            if pam_size % 2 != 0:
                pam_size += 1
            else:
                pam_size += 2

    max_distance = 0.0
    for d in np.arange(0, max_energy, tolerance):
        real_axis_pam = pam(pam_size, d)
        if real_axis_pam.max() <= max_energy:
            max_distance = d
        else:
            return max_distance
    return max_distance
    

# Performs the QAM encoding (in a linear fashion for the moment => might be re-worked)
def m_qam_encoder(codeword, constellation, alphabet, energy_per_symbol=0):
    try:
        k = alphabet.index(codeword)
        m = len(alphabet)
        return constellation[k]
    except OverflowError as ovferr:
        raise

## Encoder part

In [6]:
# Encodes a string into a sequence of complex numbers & adds dummy points in front of it, given an encoder, a constellation, an alphabet and a number of dummy points
def encode_string(raw_str, n_dummy_symbols, encoder, dummy_symbol, constellation = [], alphabet = []):
    if n_dummy_symbols >= 100:
        raise OverflowError("ERROR: The batch of dummy samples cannot be equal or exceed 100 symbols.") 
    bit_str = ascii_str_to_binary(raw_str)
    splitted_bit_str = split_bit_str(bit_str, int(np.log2(len(alphabet))))
    splitted_bit_str_size = len(splitted_bit_str)
    if splitted_bit_str_size > 100:
        raise OverflowError("ERROR: The string without the dummy symbols cannot exceed 100 symbols.") 

    if splitted_bit_str_size + n_dummy_symbols > 100:
        raise OverflowError(f"ERROR: the number of dummy samples is too high, the max value you can set here is {100 - splitted_bit_str_size}.")
            
    theta_estimator_batch = np.full((n_dummy_symbols, 1), dummy_symbol)

    return np.append(theta_estimator_batch, np.array([encoder(codeword=codeword, constellation=constellation, alphabet=alphabet) for codeword in splitted_bit_str]))

## Channel part (given)

In [7]:
def channel(sent_signal):
    s = np.mean(np.absolute(sent_signal)**2)
    if s <= 1:
        s = 1
    noise_power = (10**(-2.65))*s
    shift = np.exp(-2j*np.pi*np.random.rand())
    sent_signal = sent_signal*shift
    noise_std = np.sqrt(noise_power/2)
    rcv_signal = sent_signal + noise_std*np.random.randn(len(sent_signal)) + 1j*noise_std*np.random.randn(len(sent_signal))
    return rcv_signal

## Decoder part

In [8]:
# Decodes a bitstring outputted by the channel, given a constellation, an alphabet, a dummy symbol and the number of sent dummy symbols.

def decode_str(noisy_str, dummy_symbol, n_dummy_symbols, alphabet, constellation):
    # Isolate the dummy symbols to estimate the phase shift
    dummy_symbols = noisy_str[0:n_dummy_symbols]
    
    # Estimate the phase shift (np.angle gives a value between (-pi, pi])
    phase = (np.angle(np.sum(dummy_symbols)) + np.pi) - (np.angle(dummy_symbol) + np.pi) 

    # print(f"Estimated phase: {phase}, phase with normalization: {phase % 2*np.pi}")
    # Apply the inverse phase shift to every codeword
    dephased_symbols = np.exp(-1j * phase) * noisy_str[n_dummy_symbols:]

    decoded_codewords = [alphabet[minimum_distance_criterion(constellation, symbol)] for symbol in dephased_symbols]
    resulting_bit_str = "".join(decoded_codewords)
    return resulting_bit_str

## Putting everything together

In [10]:
max_energy = 3/2
encoder = m_qam_encoder
dummy_symbol = max_energy + 1j * max_energy

# TODO: PLAY WITH THE VALUES HERE
nb_strings = 100
distance_margin = 0.001

set_of_random_strings = [generate_ascii_strings() for i in range(0, nb_strings)]
range_of_dummy_symbols = range(0, 23) # LIMIT FOR 64-QAM IS 9 SYMBOLS, LIMIT FOR 128-QAM IS 22 SYMBOLS -> MAKES NO SENSE TO GO BEYOND 23

histogram_per_qam = {}

for str_to_process in set_of_random_strings:
    original_bitstr = ascii_str_to_binary(str_to_process)

    for codeword_size in compute_codeword_lengths(original_bitstr):
        if 2 <= codeword_size and codeword_size <= 10:
            histogram = []

            qam_size = 2**codeword_size
            d = get_max_qam_distance(qam_size, distance_margin, max_energy)
            constellation = m_qam(qam_size, d)
            alphabet = get_alphabet_from_codeword_length(codeword_size)

            if len(constellation) > 0:
                for n_dummy_symbols in range_of_dummy_symbols:
                    try:
                        # We encode the string
                        encoded_str = encode_string(
                            str_to_process, 
                            constellation=constellation, 
                            alphabet=alphabet, 
                            n_dummy_symbols=n_dummy_symbols, 
                            dummy_symbol=dummy_symbol, 
                            encoder=encoder
                        )

                        # We pass it through the channel
                        noisy_str = channel(encoded_str)

                        # We decode the string
                        decoded_bitstr = decode_str(
                            noisy_str, 
                            dummy_symbol=dummy_symbol, 
                            n_dummy_symbols=n_dummy_symbols, 
                            alphabet=alphabet,
                            constellation=constellation
                        )

                        # We compute all the positions where there are errors in the decoding process
                        diff_positions = get_diff_positions(original_bitstr, decoded_bitstr)
                        # We divide the number of bit errors by the total length of the bitstring => better metric for the histogram
                        histogram.append(len(diff_positions) / len(original_bitstr))

                    except OverflowError as ovferr: # Go to another QAM configuration when we reach/go past the max-energy limit
                        break
                    except UnicodeDecodeError as unicodeerr:
                        continue
            
            key = f"{qam_size}"
            if key in histogram_per_qam:
                histogram_per_qam[key] = [histogram_per_qam[key][i] + histogram[i] for i in range(0, len(histogram))]
            else:
                histogram_per_qam[key] = histogram

print(f"Histogram of errors per QAM configuration, for for n_dummy_symbols ranging from 0 to 22:")
print(histogram_per_qam)
print()
for key, value in histogram_per_qam.items():
    print(f"Probability of error for {key}-QAM and each value of n_dummy_symbols -> {[errors/nb_strings for errors in value]}")

# x = [ele.real for ele in data]
# y = [ele.imag for ele in data]
  
# # Plot the constellation
# plt.scatter(x, y)
# plt.ylabel('Imaginary')
# plt.xlabel('Real')
# plt.show()

Histogram of errors per QAM configuration, for for n_dummy_symbols ranging from 0 to 22:
{'4': [], '8': [], '64': [47.289377289377285, 0.005494505494505495, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003663003663003663, 0.0], '128': [48.71794871794872, 0.20146520146520144, 0.14285714285714282, 0.12454212454212453, 0.12820512820512817, 0.10073260073260071, 0.11355311355311352, 0.13553113553113547, 0.13186813186813182, 0.2014652014652014, 0.21428571428571422, 0.16483516483516486, 0.2197802197802197, 0.21062271062271054, 0.21978021978021964, 0.29304029304029294, 0.19047619047619038, 0.25091575091575086, 0.23992673992673985, 0.24908424908424898, 0.36263736263736246, 0.31868131868131844, 0.3919413919413917]}

Probability of error for 4-QAM and each value of n_dummy_symbols -> []
Probability of error for 8-QAM and each value of n_dummy_symbols -> []
Probability of error for 64-QAM and each value of n_dummy_symbols -> [0.4728937728937728, 5.494505494505495e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.6630036630